In [18]:
import pandas as pd

# Load the Excel file
excel_path = "PR.0079.ALMA for Stores, RH, Product.xlsx"

# Step 1: Load and clean the "PLANEAMENTO" sheet
planeamento_data = pd.read_excel(excel_path, sheet_name="PLANEAMENTO", header=None)
header_row_idx = planeamento_data[planeamento_data.apply(lambda row: row.str.contains("FASES/TAREFAS", na=False).any(), axis=1)].index[0]
initial_cleaned_data = pd.read_excel(excel_path, sheet_name="PLANEAMENTO", header=header_row_idx)
initial_cleaned_data = initial_cleaned_data.dropna(how="all").reset_index(drop=True)
print("Initial cleaned data:")
print(initial_cleaned_data)

Initial cleaned data:
    Unnamed: 0 Unnamed: 1            EDT                     FASES/TAREFAS  \
0          NaN          P        PR.0079    ALMA for Stores / RH / Product   
1          1.0          F      PR.0079.1                 Research and Plan   
2          1.0          T    PR.0079.1.1           Speak with Service Desk   
3          2.0          T    PR.0079.1.2   Speak with Store communications   
4          3.0          T    PR.0079.1.3          Speak with store manager   
..         ...        ...            ...                               ...   
66         7.0          T   PR.0079.12.7                Release in batch 3   
67         8.0          M   PR.0079.12.8   MILESTONE: All batches released   
68         9.0          T   PR.0079.12.9  Review engagement in all Batches   
69        10.0          T  PR.0079.12.10     Write an list of improvements   
70        11.0          M  PR.0079.12.11          MILESTONE: PROJECT ENDED   

                               RESPONSÁVE

/Users/joaomontenegro/Documents/GitHub/notion_automator/myenv/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/Users/joaomontenegro/Documents/GitHub/notion_automator/myenv/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [19]:
# Step 2: Determine "Type" (Fase, Tarefa, or Milestone) and identify parent tasks
data_with_types = initial_cleaned_data.copy()
data_with_types["Type"] = data_with_types.apply(
    lambda row: "Fase" if pd.notna(row["FASES/TAREFAS"]) and pd.isna(row["RESPONSÁVEL"]) else (
        "Milestone" if pd.isna(row["FASES/TAREFAS"]) else "Tarefa"
    ),
    axis=1,
)
data_with_types["Parent Task"] = None
parent_stack = []

for idx, row in data_with_types.iterrows():
    if row["Type"] == "Fase":
        parent_stack = [row["FASES/TAREFAS"]]
    elif row["Type"] == "Tarefa" or row["Type"] == "Milestone":
        if parent_stack:
            data_with_types.at[idx, "Parent Task"] = parent_stack[-1]
    print(f"Processed row {idx}: {row}")

print("Data with types and parent tasks:")
print(data_with_types)

Processed row 0: Unnamed: 0                                         NaN
Unnamed: 1                                           P
EDT                                            PR.0079
FASES/TAREFAS           ALMA for Stores / RH / Product
RESPONSÁVEL                                        NaN
INÍCIO                             2024-01-08 00:00:00
DURAÇÃO (Dias)                                   191.0
TRABALHO TOTAL                                   231.0
TRABALHO ESPERADO                                231.0
FIM                                2024-09-30 00:00:00
DEPENDÊNCIAS                                       NaN
INÍCIO.1                           2024-01-08 00:00:00
TRABALHO REALIZADO                               231.0
TRABALHO PENDENTE                                  0.0
DURAÇÃO (Dias).1                                 428.0
DATA FIM (Calculada)               2024-11-15 00:00:00
DATA FIM                           2024-11-15 00:00:00
TIPO DE DATA                                    

In [20]:
# Step 3: Handle planned and real dates
date_cleaned_data = data_with_types.copy()
planned_start_col = [col for col in date_cleaned_data.columns if "INÍCIO" in col][0]
real_start_col = [col for col in date_cleaned_data.columns if "INÍCIO" in col][1]
planned_end_col = [col for col in date_cleaned_data.columns if "FIM" in col][0]
real_end_col = [col for col in date_cleaned_data.columns if "FIM" in col][1]

date_cleaned_data.rename(
    columns={planned_start_col: "Planned Start", real_start_col: "Real Start",
             planned_end_col: "Planned End", real_end_col: "Real End"},
    inplace=True,
)

# Generate "Datas planeadas" and "Datas reais" in the format "Start Date → End Date"
date_cleaned_data["Datas planeadas"] = date_cleaned_data.apply(
    lambda row: f"{row['Planned Start']} → {row['Planned End']}" if pd.notna(row["Planned End"]) else f"{row['Planned Start']}",
    axis=1,
)
date_cleaned_data["Datas reais"] = date_cleaned_data.apply(
    lambda row: f"{row['Real Start']} → {row['Real End']}" if pd.notna(row["Real End"]) else f"{row['Real Start']}",
    axis=1,
)
print("Data after handling dates:")
print(date_cleaned_data)

Data after handling dates:
    Unnamed: 0 Unnamed: 1            EDT                     FASES/TAREFAS  \
0          NaN          P        PR.0079    ALMA for Stores / RH / Product   
1          1.0          F      PR.0079.1                 Research and Plan   
2          1.0          T    PR.0079.1.1           Speak with Service Desk   
3          2.0          T    PR.0079.1.2   Speak with Store communications   
4          3.0          T    PR.0079.1.3          Speak with store manager   
..         ...        ...            ...                               ...   
66         7.0          T   PR.0079.12.7                Release in batch 3   
67         8.0          M   PR.0079.12.8   MILESTONE: All batches released   
68         9.0          T   PR.0079.12.9  Review engagement in all Batches   
69        10.0          T  PR.0079.12.10     Write an list of improvements   
70        11.0          M  PR.0079.12.11          MILESTONE: PROJECT ENDED   

                               RESPO

In [21]:
# Step 4: Remove metadata rows and keep valid tasks and milestones
filtered_data = date_cleaned_data.copy()
metadata_criteria = (filtered_data["FASES/TAREFAS"] == filtered_data["FASES/TAREFAS"]) & (filtered_data["RESPONSÁVEL"].isna())
filtered_data = filtered_data[~metadata_criteria].reset_index(drop=True)
print("Data after filtering metadata rows:")
print(filtered_data)

Data after filtering metadata rows:
    Unnamed: 0 Unnamed: 1            EDT  \
0          1.0          T    PR.0079.1.1   
1          2.0          T    PR.0079.1.2   
2          3.0          T    PR.0079.1.3   
3          4.0          T    PR.0079.1.4   
4          1.0          T    PR.0079.2.1   
5          2.0          T    PR.0079.2.2   
6          3.0          T    PR.0079.2.3   
7          4.0          T    PR.0079.2.4   
8          5.0          T    PR.0079.2.5   
9          1.0          T    PR.0079.3.1   
10         2.0          T    PR.0079.3.2   
11         1.0          T    PR.0079.4.1   
12         1.0          T    PR.0079.5.1   
13         2.0          T    PR.0079.5.2   
14         3.0          T    PR.0079.5.3   
15         4.0          T    PR.0079.5.4   
16         1.0          T    PR.0079.6.1   
17         2.0          T    PR.0079.6.2   
18         3.0          T    PR.0079.6.3   
19         1.0          T    PR.0079.7.1   
20         1.0          T    PR.0079.8.1

In [22]:
# Step 5: Create Notion-ready structure
notion_structure = pd.DataFrame(columns=[
    "Tarefa", "Type", "Parent Task", "Datas planeadas", "Datas reais"
])

# Populate the Notion structure
notion_structure["Tarefa"] = filtered_data["FASES/TAREFAS"]
notion_structure["Type"] = filtered_data["Type"]
notion_structure["Parent Task"] = filtered_data["Parent Task"]
notion_structure["Datas planeadas"] = filtered_data["Datas planeadas"]
notion_structure["Datas reais"] = filtered_data["Datas reais"]

# Display the final Notion-ready structure
print("Final Notion-ready structure:")
print(notion_structure)

Final Notion-ready structure:
                                               Tarefa    Type  \
0                             Speak with Service Desk  Tarefa   
1                     Speak with Store communications  Tarefa   
2                            Speak with store manager  Tarefa   
3                                      Write v.1 Plan  Tarefa   
4                Explore technical stack to implement  Tarefa   
5          Implement selected features for v.1 (demo)  Tarefa   
6              Test performance internally v.1 (demo)  Tarefa   
7                    Select participants in demo test  Tarefa   
8                                   Write test script  Tarefa   
9   Conduct internal user test with selected parti...  Tarefa   
10                                  Write test report  Tarefa   
11                        Finalize Plan for v.5 (MVP)  Tarefa   
12          Implement selected features for v.5 (MVP)  Tarefa   
13              Test performance internally v.5 (MVP)  Taref